In [2]:
# Import necessary libraries
import pandas as pd
from collections import Counter
import re

# Load the dataset
file_path = 'merged_starbucks_reviews_with_state.parquet'
df = pd.read_parquet(file_path)

# Filter negative reviews
negative_reviews = df[df['avg_rating'] < 3]['text'].dropna()

# Define patterns to search for
patterns = {
    'slow service': ['slow', 'long', 'line', 'slowest'],
    'rude staff': ['rude', 'bad', 'unfriendly', 'horrible', 'customer', 'nasty'],
    'incorrect orders': ['wrong', 'size', 'messed', 'incorrect'],
    'lack of product availability': ['stock', 'no cups', 'no lids', 'no ingredients', 'ran out'],
    'high prices': ['expensive', 'price', 'overpriced', 'expensive']
}

# Count the frequency of each pattern
pattern_counts = Counter()
for review in negative_reviews:
    for pattern, keywords in patterns.items():
        if any(re.search(keyword, review, re.IGNORECASE) for keyword in keywords):
            pattern_counts[pattern] += 1

# Check if there are elements in the counter before trying to access them
if pattern_counts:
    most_common_pattern = pattern_counts.most_common(1)[0]
    print('The most common pattern is:', most_common_pattern[0])
    print('Frequency:', most_common_pattern[1])
else:
    print('No common patterns found.')

The most common pattern is: slow service
Frequency: 121


In [3]:
# Contar la frecuencia de la palabra 'slow' en la columna 'text'
slow_count = df['text'].str.count('slow', flags=re.IGNORECASE).sum()
print('La palabra "slow" se repite:', slow_count, 'veces en la columna "text".')

La palabra "slow" se repite: 2283.0 veces en la columna "text".


- **MODELOS DE PREDICCION RNNs y CNNs.** 

- Medimos que la puntuación promedio de Starbucks sea superior en un 10% a la de su principal competidor(dunkin), medida por semestre(entre el ultimo semestre y el semestre futuro).

RNNs:

In [2]:


import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from sklearn.model_selection import train_test_split

# Cargar los datos
starbucks_df = pd.read_parquet('../data/Starbucks_reviews_ETL_limpio.parquet')
dunkin_df = pd.read_parquet('../data/Dunkin_reviews_ETL_limpio.parquet')

# Preparar los datos
X_starbucks = starbucks_df['rating'].values.reshape(-1, 1)
y_starbucks = starbucks_df['rating'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_starbucks, y_starbucks, test_size=0.2, random_state=42)

# Definir el modelo RNN
model_rnn = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(1, 1)),
    Dense(1)
])

# Compilar el modelo
model_rnn.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model_rnn.fit(X_train, y_train, epochs=10, verbose=1)

# Evaluar el modelo
loss_rnn = model_rnn.evaluate(X_test, y_test)

# Hacer predicciones para el semestre futuro
next_semester_prediction_rnn = model_rnn.predict(np.array([[4.0]]))  # Usamos 4.0 como ejemplo

# Comparar con la puntuación promedio de Dunkin en el último semestre
dunkin_avg_last_semester = dunkin_df['rating'].mean()

# Calcular la diferencia porcentual y absoluta
percentage_difference_rnn = ((next_semester_prediction_rnn[0][0] - dunkin_avg_last_semester) / dunkin_avg_last_semester) * 100
absolute_difference_rnn = dunkin_avg_last_semester - next_semester_prediction_rnn[0][0]

print('Pérdida en el conjunto de prueba (RNN):', loss_rnn)
print('Predicción para el próximo semestre (RNN):', next_semester_prediction_rnn[0][0])
print('Diferencia porcentual entre la predicción del modelo RNN y la puntuación de Dunkin en el último semestre:', percentage_difference_rnn)
print('Diferencia absoluta en puntuaciones (RNN):', absolute_difference_rnn)

Epoch 1/10


c:\Users\zimme\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 590us/step - loss: 2.6842
Epoch 2/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 590us/step - loss: 2.7250e-06
Epoch 3/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 588us/step - loss: 6.9828e-09
Epoch 4/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 585us/step - loss: 4.8784e-14
Epoch 5/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 599us/step - loss: 0.0000e+00
Epoch 6/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 596us/step - loss: 0.0000e+00
Epoch 7/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 601us/step - loss: 0.0000e+00
Epoch 8/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 598us/step - loss: 0.0000e+00
Epoch 9/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 595us/step - loss: 0.0000e+00
Epoch 10/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 594us/step - loss: 0.0000e+00
980/980 ━━━━━━━━━━━━━━━━━━━━ 1s 422us/step - loss: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Pérdida en el conjunto de prueba (RNN): 0.0
Predicción para el próximo semestre (RNN): 4.0
Diferencia porcentual entre la predicción del modelo RNN y la puntuación 

In [3]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from sklearn.model_selection import train_test_split

# Preparar los datos para Starbucks
X_starbucks = starbucks_df['rating'].values.reshape(-1, 1)
y_starbucks = starbucks_df['rating'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_starbucks, X_test_starbucks, y_train_starbucks, y_test_starbucks = train_test_split(X_starbucks, y_starbucks, test_size=0.2, random_state=42)

# Definir el modelo RNN para Starbucks
model_rnn_starbucks = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(1, 1)),
    Dense(1)
])

# Compilar el modelo
model_rnn_starbucks.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model_rnn_starbucks.fit(X_train_starbucks, y_train_starbucks, epochs=10, verbose=1)

# Hacer predicciones para el semestre futuro de Starbucks
next_semester_prediction_rnn_starbucks = model_rnn_starbucks.predict(np.array([[4.0]]))  # Usamos 4.0 como ejemplo

# Preparar los datos para Dunkin
X_dunkin = dunkin_df['rating'].values.reshape(-1, 1)
y_dunkin = dunkin_df['rating'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_dunkin, X_test_dunkin, y_train_dunkin, y_test_dunkin = train_test_split(X_dunkin, y_dunkin, test_size=0.2, random_state=42)

# Definir el modelo RNN para Dunkin
model_rnn_dunkin = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(1, 1)),
    Dense(1)
])

# Compilar el modelo
model_rnn_dunkin.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model_rnn_dunkin.fit(X_train_dunkin, y_train_dunkin, epochs=10, verbose=1)

# Hacer predicciones para el semestre futuro de Dunkin
next_semester_prediction_rnn_dunkin = model_rnn_dunkin.predict(np.array([[4.0]]))  # Usamos 4.0 como ejemplo

print('Predicción para el próximo semestre (RNN) de Starbucks:', next_semester_prediction_rnn_starbucks[0][0])
print('Predicción para el próximo semestre (RNN) de Dunkin:', next_semester_prediction_rnn_dunkin[0][0])

Epoch 1/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 587us/step - loss: 2.3290
Epoch 2/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 591us/step - loss: 2.4437e-06
Epoch 3/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 591us/step - loss: 1.9845e-07
Epoch 4/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 589us/step - loss: 4.1966e-07
Epoch 5/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 583us/step - loss: 2.0151e-06
Epoch 6/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 590us/step - loss: 1.3163e-06
Epoch 7/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 594us/step - loss: 1.0273e-06
Epoch 8/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 603us/step - loss: 2.0304e-06
Epoch 9/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 593us/step - loss: 1.0907e-06
Epoch 10/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 2s 589us/step - loss: 2.5278e-06
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Epoch 1/10
2898/2898 ━━━━━━━━━━━━━━━━━━━━ 2s 592us/step - loss: 1.9767
Epoch 2/10
2898/2898 ━━━━━━━━━━━━━━━━━━━━ 2s 587us/step - loss: 7.9654e-05
Epoch 3/10
2898/2898 ━━━━━━━━━━━━━━━━━━━━ 2s 591us/step - loss: 9.562

-CNNs:

In [5]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from sklearn.model_selection import train_test_split

# Preparar los datos
X_starbucks = starbucks_df['rating'].values.reshape(-1, 1)
y_starbucks = starbucks_df['rating'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_starbucks, y_starbucks, test_size=0.2, random_state=42)

# Definir el modelo CNN
model_cnn = Sequential([
    Conv1D(64, kernel_size=1, activation='relu', input_shape=(1, 1)),  # Cambio de kernel_size a 1
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compilar el modelo
model_cnn.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model_cnn.fit(X_train, y_train, epochs=10, verbose=1)

# Evaluar el modelo
loss_cnn = model_cnn.evaluate(X_test, y_test)

# Hacer predicciones para el semestre futuro
next_semester_prediction_cnn = model_cnn.predict(np.array([[4.0]]))  # Usamos 4.0 como ejemplo

# Comparar con la puntuación promedio de Dunkin en el último semestre
dunkin_avg_last_semester = dunkin_df['rating'].mean()

# Calcular la diferencia porcentual y absoluta
percentage_difference_cnn = ((next_semester_prediction_cnn[0][0] - dunkin_avg_last_semester) / dunkin_avg_last_semester) * 100
absolute_difference_cnn = dunkin_avg_last_semester - next_semester_prediction_cnn[0][0]

print('Pérdida en el conjunto de prueba (CNN):', loss_cnn)
print('Predicción para el próximo semestre (CNN):', next_semester_prediction_cnn[0][0])
print('Diferencia porcentual entre la predicción del modelo CNN y la puntuación de Dunkin en el último semestre:', percentage_difference_cnn)
print('Diferencia absoluta en puntuaciones (CNN):', absolute_difference_cnn)

Epoch 1/10


c:\Users\zimme\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 685us/step - loss: 0.3502
Epoch 2/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 680us/step - loss: 7.7069e-13
Epoch 3/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 679us/step - loss: 3.5066e-06
Epoch 4/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 677us/step - loss: 1.6046e-05
Epoch 5/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 681us/step - loss: 1.3463e-05
Epoch 6/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 689us/step - loss: 8.0874e-06
Epoch 7/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 681us/step - loss: 9.9040e-06
Epoch 8/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 687us/step - loss: 1.2647e-05
Epoch 9/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 685us/step - loss: 1.4092e-05
Epoch 10/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 689us/step - loss: 8.7441e-06
980/980 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - loss: 6.6780e-05
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Pérdida en el conjunto de prueba (CNN): 6.693222530884668e-05
Predicción para el próximo semestre (CNN): 3.992558
Diferencia porcentual entre la predicción del mode

In [6]:
from tensorflow.keras.layers import Conv1D, Flatten

# Definir el modelo CNN para Starbucks
model_cnn_starbucks = Sequential([
    Conv1D(64, kernel_size=1, activation='relu', input_shape=(1, 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compilar el modelo
model_cnn_starbucks.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model_cnn_starbucks.fit(X_train_starbucks, y_train_starbucks, epochs=10, verbose=1)

# Hacer predicciones para el semestre futuro de Starbucks
next_semester_prediction_cnn_starbucks = model_cnn_starbucks.predict(np.array([[4.0]]))  # Usamos 4.0 como ejemplo

# Definir el modelo CNN para Dunkin
model_cnn_dunkin = Sequential([
    Conv1D(64, kernel_size=1, activation='relu', input_shape=(1, 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compilar el modelo
model_cnn_dunkin.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model_cnn_dunkin.fit(X_train_dunkin, y_train_dunkin, epochs=10, verbose=1)

# Hacer predicciones para el semestre futuro de Dunkin
next_semester_prediction_cnn_dunkin = model_cnn_dunkin.predict(np.array([[4.0]]))  # Usamos 4.0 como ejemplo

print('Predicción para el próximo semestre (CNN) de Starbucks:', next_semester_prediction_cnn_starbucks[0][0])
print('Predicción para el próximo semestre (CNN) de Dunkin:', next_semester_prediction_cnn_dunkin[0][0])

Epoch 1/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 678us/step - loss: 0.4866
Epoch 2/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 681us/step - loss: 3.7530e-08
Epoch 3/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 699us/step - loss: 4.7638e-06
Epoch 4/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 689us/step - loss: 1.1228e-05
Epoch 5/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 682us/step - loss: 1.1547e-05
Epoch 6/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 671us/step - loss: 1.4433e-05
Epoch 7/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 678us/step - loss: 1.1606e-05
Epoch 8/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 668us/step - loss: 1.4721e-05
Epoch 9/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 674us/step - loss: 1.3086e-05
Epoch 10/10
3919/3919 ━━━━━━━━━━━━━━━━━━━━ 3s 675us/step - loss: 1.3066e-05
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Epoch 1/10
2898/2898 ━━━━━━━━━━━━━━━━━━━━ 2s 675us/step - loss: 0.4260
Epoch 2/10
2898/2898 ━━━━━━━━━━━━━━━━━━━━ 2s 684us/step - loss: 1.0208e-14
Epoch 3/10
2898/2898 ━━━━━━━━━━━━━━━━━━━━ 2s 695us/step - loss: 8.733

- Promedios de cada compañia en el ultimo semestre.

In [7]:
import pandas as pd


# Filtrar los datos del último semestre
# Asumiendo que la columna 'date' contiene las fechas y está en formato datetime
starbucks_df['date'] = pd.to_datetime(starbucks_df['date'])
dunkin_df['date'] = pd.to_datetime(dunkin_df['date'])

# Obtener la fecha más reciente en los datos
max_date_starbucks = starbucks_df['date'].max()
max_date_dunkin = dunkin_df['date'].max()

# Calcular la fecha de inicio del último semestre
start_date_starbucks = max_date_starbucks - pd.DateOffset(months=6)
start_date_dunkin = max_date_dunkin - pd.DateOffset(months=6)

# Filtrar los datos del último semestre
last_semester_starbucks = starbucks_df[starbucks_df['date'] >= start_date_starbucks]
last_semester_dunkin = dunkin_df[dunkin_df['date'] >= start_date_dunkin]

# Calcular los promedios
average_starbucks = last_semester_starbucks['rating'].mean()
average_dunkin = last_semester_dunkin['rating'].mean()

print('Promedio de Starbucks en el último semestre:', average_starbucks)
print('Promedio de Dunkin en el último semestre:', average_dunkin)

Promedio de Starbucks en el último semestre: 4.28763353836193
Promedio de Dunkin en el último semestre: 4.065958576739246


-Diferencias porcentuales de las predicciones de los ratings de ambas compañías para los dos modelos (RNNs y CNNs) con los promedios de cada compañía en el último semestre

In [8]:
# Predicciones de los modelos
pred_rnn_starbucks = 4.0
pred_rnn_dunkin = 4.0
pred_cnn_starbucks = 4.0
pred_cnn_dunkin = 4.000641

# Promedios del último semestre
avg_starbucks = 4.28763353836193
avg_dunkin = 4.065958576739246

# Calcular las diferencias porcentuales
perc_diff_rnn_starbucks = ((pred_rnn_starbucks - avg_starbucks) / avg_starbucks) * 100
perc_diff_rnn_dunkin = ((pred_rnn_dunkin - avg_dunkin) / avg_dunkin) * 100
perc_diff_cnn_starbucks = ((pred_cnn_starbucks - avg_starbucks) / avg_starbucks) * 100
perc_diff_cnn_dunkin = ((pred_cnn_dunkin - avg_dunkin) / avg_dunkin) * 100

print('Diferencia porcentual (RNN) de Starbucks:', perc_diff_rnn_starbucks)
print('Diferencia porcentual (RNN) de Dunkin:', perc_diff_rnn_dunkin)
print('Diferencia porcentual (CNN) de Starbucks:', perc_diff_cnn_starbucks)
print('Diferencia porcentual (CNN) de Dunkin:', perc_diff_cnn_dunkin)

Diferencia porcentual (RNN) de Starbucks: -6.708445014911857
Diferencia porcentual (RNN) de Dunkin: -1.6222146756876776
Diferencia porcentual (CNN) de Starbucks: -6.708445014911857
Diferencia porcentual (CNN) de Dunkin: -1.6064496355894593


- Cumplimiento del KPI.

In [9]:
# Calcular el 10% del promedio de Dunkin para el próximo semestre (predicción RNN)
threshold_rnn_next_semester = pred_rnn_dunkin * 1.10

# Comparar con la predicción de Starbucks para el próximo semestre (predicción RNN)
is_above_threshold_rnn_next_semester = pred_rnn_starbucks > threshold_rnn_next_semester

# Calcular el 10% del promedio de Dunkin para el próximo semestre (predicción CNN)
threshold_cnn_next_semester = pred_cnn_dunkin * 1.10

# Comparar con la predicción de Starbucks para el próximo semestre (predicción CNN)
is_above_threshold_cnn_next_semester = pred_cnn_starbucks > threshold_cnn_next_semester

print('¿El promedio de Starbucks es superior en un 10% al de Dunkin para el próximo semestre (RNN)?', is_above_threshold_rnn_next_semester)
print('¿El promedio de Starbucks es superior en un 10% al de Dunkin para el próximo semestre (CNN)?', is_above_threshold_cnn_next_semester)

¿El promedio de Starbucks es superior en un 10% al de Dunkin para el próximo semestre (RNN)? False
¿El promedio de Starbucks es superior en un 10% al de Dunkin para el próximo semestre (CNN)? False
